# Variational AutoEncoder (VAE)

In [ ]:
import torch
from torch import nn
from torch import optim

import torchvision
import torchvision.transforms as transforms

In [ ]:
# work on GPUs?

In [ ]:
# import Dataset (MNIST)
transform = transforms.Compose([
    transforms.ToTensor(),                             
    # <-- we dont normalize here, can you guess why?
])

train_dataset = torchvision.datasets.MNIST(root='./Data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./Data', train=False, download=True, transform=transform)

BATCH_SIZE = 128
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
# build model
class VAE(nn.Module):
    def __init__(self, input_dim, out_dim, latent_dim):
        super().__init__()
        self.flatten = nn.Flatten(1,3)
        self.layer_1 = nn.Linear(input_dim, latent_dim)
        self.activation = nn.ReLU()
        self.layer_2 = nn.Linear(latent_dim, out_dim)
        self.soft = nn.Softmax(dim=1)

    def encode(self, x):
        x = self.flatten(x)
        x = self.layer_1(x)
        z = self.activation(x)

        return z

    def decode(self, z):
        z = self.layer_2(z)
        x = self.soft(z)

        return x

latent_dim = 2
model = VAE(28*28, 28*28, latent_dim)
# model = model.to(device)

In [ ]:
# train loop

In [ ]:
# evaluation and visualization